In [ ]:
import requests
import json
import os
from fpdf import FPDF

# Ollama API configuration
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:latest"

# Directory to save generated content
OUTPUT_DIR = "book_content_detail"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_toc(subject):
    """Generate a Table of Contents for the given subject."""
    prompt = f"""
    You are an expert assistant skilled at creating detailed book outlines.
    Given a subject, your task is to create a comprehensive Table of Contents for a book on the given subject.
    
    Each chapter should have sections, and when relevant, sections should have subsubsections. 
    Structure the Table of Contents hierarchically to ensure maximum detail and granularity.
    
    Now generate a detailed Table of Contents for the book with Subject: {subject}.
    
    Structure the response as a valid JSON object:
    {{
        "Table_of_Contents": [
            {{
                "Title": "Chapter 1: Title of Chapter",
                "Sections": [
                    {{
                        "Title": "Section 1.1: Title of Section",
                        "Subsections": [
                            "Subsubsection 1.1.1: Title of Subsubsection",
                            "Subsubsection 1.1.2: Title of Subsubsection"
                        ]
                    }},
                    {{
                        "Title": "Section 1.2: Title of Section",
                        "Subsections": []
                    }}
                ]
            }},
            {{
                "Title": "Chapter 2: Title of Chapter",
                "Sections": []
            }}
        ]
    }}
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    content = response.json().get("response", "")
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        raise Exception(f"Failed to parse JSON response: {content}")

def generate_section(title):
    """Generate content for a given chapter, section, or subsubsection title."""
    prompt = f"""
    You are a knowledgeable assistant tasked with writing detailed content for a book.
    Please write content for the following title:
    Title: '{title}'
    
    The content should be comprehensive, well-structured, and professional.
    Include examples, subtopics, and technical insights where applicable. 
    Expand on subtopics wherever possible to provide a deep understanding of the subject.
    """
    payload = {"model": MODEL_NAME, "prompt": prompt, "stream": False}
    response = requests.post(OLLAMA_URL, json=payload)
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    return response.json().get("response", "")

def save_content_to_file(filename, content):
    """Save generated content to a file."""
    with open(os.path.join(OUTPUT_DIR, filename), "w") as f:
        f.write(content)

def create_pdf_from_content(toc, output_file=f"{OUTPUT_DIR}/Generated_Book_Detail.pdf"):
    """Generate a PDF book from the content."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    
    # Title Page
    pdf.add_page()
    pdf.set_font("Arial", size=24, style="B")
    pdf.cell(0, 10, "Generated Book", ln=True, align="C")
    pdf.ln(20)
    
    # Table of Contents
    pdf.set_font("Arial", size=16, style="B")
    pdf.cell(0, 10, "Table of Contents", ln=True)
    pdf.ln(10)
    pdf.set_font("Arial", size=12)
    for chapter in toc["Table_of_Contents"]:
        pdf.cell(0, 10, chapter["Title"], ln=True)
        for section in chapter["Sections"]:
            pdf.cell(10, 10, f"  {section['Title']}", ln=True)
            for subsubsection in section["Subsections"]:
                pdf.cell(20, 10, f"    {subsubsection}", ln=True)
        pdf.ln(5)
    
    # Chapters
    for chapter in toc["Table_of_Contents"]:
        pdf.add_page()
        pdf.set_font("Arial", size=18, style="B")
        pdf.cell(0, 10, chapter["Title"], ln=True)
        pdf.ln(10)
        
        chapter_file = f"{chapter['Title'].replace(' ', '_')}.txt"
        chapter_path = os.path.join(OUTPUT_DIR, chapter_file)
        if os.path.exists(chapter_path):
            with open(chapter_path, "r") as f:
                content = f.read()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10, content)
        
        for section in chapter["Sections"]:
            pdf.add_page()
            pdf.set_font("Arial", size=16, style="B")
            pdf.cell(0, 10, section["Title"], ln=True)
            pdf.ln(5)
            
            section_file = f"{section['Title'].replace(' ', '_')}.txt"
            section_path = os.path.join(OUTPUT_DIR, section_file)
            if os.path.exists(section_path):
                with open(section_path, "r") as f:
                    content = f.read()
                pdf.set_font("Arial", size=12)
                pdf.multi_cell(0, 10, content)
            
            for subsubsection in section["Subsections"]:
                pdf.add_page()
                pdf.set_font("Arial", size=14, style="B")
                pdf.cell(0, 10, subsubsection, ln=True)
                pdf.ln(5)
                
                subsubsection_file = f"{subsubsection.replace(' ', '_')}.txt"
                subsubsection_path = os.path.join(OUTPUT_DIR, subsubsection_file)
                if os.path.exists(subsubsection_path):
                    with open(subsubsection_path, "r") as f:
                        content = f.read()
                    pdf.set_font("Arial", size=12)
                    pdf.multi_cell(0, 10, content)
    
    pdf.output(output_file)

# Main Execution
if __name__ == "__main__":
    subject = "Large Language Model"
    print(f"Generating Table of Contents for subject: {subject}")
    
    # Step 1: Generate ToC
    toc = generate_toc(subject)
    print("Table of Contents generated successfully.")
    save_content_to_file(f"toc.txt", json.dumps(toc, indent=4))
    
    # Step 2: Generate content for each chapter/section/subsubsection
    for chapter in toc["Table_of_Contents"]:
        chapter_title = chapter["Title"]
        print(f"Generating content for {chapter_title}...")
        chapter_content = generate_section(chapter_title)
        save_content_to_file(f"{chapter_title.replace(' ', '_')}.txt", chapter_content)
        
        for section in chapter["Sections"]:
            section_title = section["Title"]
            print(f"Generating content for {section_title}...")
            section_content = generate_section(section_title)
            save_content_to_file(f"{section_title.replace(' ', '_')}.txt", section_content)
            
            for subsubsection_title in section["Subsections"]:
                print(f"Generating content for {subsubsection_title}...")
                subsubsection_content = generate_section(subsubsection_title)
                save_content_to_file(f"{subsubsection_title.replace(' ', '_')}.txt", subsubsection_content)
    
    # Step 3: Create PDF
    print("Merging content into a PDF book...")
    create_pdf_from_content(toc)
    print(f"PDF book generated successfully at location: {OUTPUT_DIR}/Generated_Book_Detail.pdf")